# Tuning GNN-ePC-SAFT

## Starting point

In [ ]:
import os, shutil

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_1 = user_secrets.get_secret("kaggle")
secret_value_2 = user_secrets.get_secret("username")
secret_value_3 = user_secrets.get_secret("wandbkey")


os.environ["KAGGLE_USERNAME"] = secret_value_2

os.environ["KAGGLE_KEY"] = secret_value_1

os.environ["WANDB_API_KEY"] = secret_value_3

os.environ["PATH"] = "/root/.local/bin:" + os.environ["PATH"]

In [ ]:
%%bash

# for the cloud 

rm -rf .* *

git clone --branch dev --single-branch -q https://github.com/wildsonbbl/gnnepcsaft.git .


In [ ]:
%%capture req_output

%%bash

pip install -r requirements.txt

echo "*********  Finished pip installing requirements.txt  *********"

# pip install torch-scatter

echo "*********  Finished pip installing torch-scatter  *********"



# install pcsaft

curl -O -L https://gitlab.com/libeigen/eigen/-/archive/master/eigen-master.zip
curl -O -L https://github.com/zmeri/PC-SAFT/archive/refs/tags/v1.5.0.zip
unzip -q eigen-master.zip
unzip -q v1.5.0.zip
sed -i "s/np.float_/np.float64/g" PC-SAFT-1.5.0/pcsaft.pyx 
cp -rf eigen-master/. PC-SAFT-1.5.0/externals/eigen
rm -rf eigen-master v1.5.0.zip eigen-master.zip
pip install ./PC-SAFT-1.5.0

# install pcsaft

echo "*********  Finished pip installing pcsaft  *********"


In [ ]:
!echo '{req_output}' > log.txt

In [ ]:
cp -r /kaggle/input/gnnepcsaft/* gnnepcsaft/data

In [ ]:
# previous running results
# !unzip /kaggle/working/tuning.zip
# !cp -a /kaggle/working/root/* ~/
# !rm -rf /kaggle/working/root

In [ ]:
%%bash

pip show pcsaft | grep -E "Version|Name"



lscpu | grep -E "Model name|Core|core" 



nvidia-smi

## Tuning

### Tuning run

In [ ]:
%%bash

python -m gnnepcsaft.train.tuner \
--workdir=/kaggle/working/gnnepcsaft --config=gnnepcsaft/configs/default.py \
--tags="batch 1","v1.1" \
--num_cpu=0.75 --num_cpu_trainer=0.25 --num_gpus=0.25 \
--num_samples=64 --time_budget_s=39600 --verbose=0 \
--config.num_train_steps=25000 --config.eval_every_steps=5000 \
--config.model=PNAL --config.batch_size=512 \
--config.job_type="tuning" \
--get_result=False #--restoredir=/root/ray_results/gnnpcsaft/

### Zip results

In [ ]:
%%bash
rm -rf ~/ray*/gnn*/Torch*/check* ~/ray*/gnn*/Torch*/wandb 
zip -r tuning.zip ~/ray* -q